# 05 · Validação de Correção de Leakage
## Teste de Integridade Pós-Correção

<div align="center">

```
┌─────────────────────────────────────────────────────────────┐
│   LEAKAGE CORRECTION VALIDATION - INTEGRITY TEST v1.0      │
└─────────────────────────────────────────────────────────────┘
```

![Status](https://img.shields.io/badge/Status-Validation-blue)
![Priority](https://img.shields.io/badge/Priority-CRITICAL-red)
![Type](https://img.shields.io/badge/Type-Quality%20Assurance-success)

</div>

---

### OBJETIVO

Validar se as correções aplicadas no notebook 03 (feature audit) eliminaram o data leakage, garantindo que o modelo possa generalizar para dados não vistos.

### PROBLEMA A SER VALIDADO

<div style="background-color: #2d1a1a; border-left: 4px solid #ef4444; padding: 15px; border-radius: 4px;">

**SITUAÇÃO ANTERIOR (COM LEAKAGE):**
```
Performance Gap Analysis (ANTES)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Training PR-AUC  : 0.999  [████████████████████] Perfect  
Test PR-AUC      : 0.007  [█                   ] Random
Gap              : 153x degradation
Adversarial AUC  : 0.89  (Severe leakage detected)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
```

</div>

### CRITÉRIOS DE SUCESSO

<table>
<tr><th>Métrica</th><th>Threshold</th><th>Interpretação</th></tr>
<tr><td>Adversarial AUC</td><td>< 0.55</td><td>Sem leakage detectável</td></tr>
<tr><td>Adversarial AUC</td><td>0.55 - 0.60</td><td>Leakage mínimo/aceitável</td></tr>
<tr><td>Adversarial AUC</td><td>> 0.65</td><td>Leakage ainda presente</td></tr>
<tr><td>Test PR-AUC</td><td>> 0.20</td><td>Modelo generaliza adequadamente</td></tr>
<tr><td>Train-Test Gap</td><td>< 0.40</td><td>Overfitting controlado</td></tr>
<tr><td>Correlation Gap</td><td>< 5x</td><td>Features consistentes</td></tr>
</table>

### ESTRATÉGIA DE VALIDAÇÃO

```
┌────────────────┐    ┌────────────────┐    ┌────────────────┐
│   Adversarial  │ -> │  Correlation   │ -> │    Model       │
│   Validation   │    │  Gap Analysis  │    │  Performance   │
└────────────────┘    └────────────────┘    └────────────────┘
         │                     │                      │
         └─────────────────────┴──────────────────────┘
                              │
                    ┌─────────▼─────────┐
                    │  Pass/Fail        │
                    │  Determination    │
                    └───────────────────┘
```

> **CHECKLIST DE VALIDAÇÃO:** Este notebook executa 4 testes independentes. Todos devem passar para considerar o leakage corrigido.

---

In [1]:
# Setup
import sys
from pathlib import Path
sys.path.insert(0, str(Path('..') / 'utils'))

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

DATA_DIR = Path('../data')
print("[OK] Setup completo")

[OK] Setup completo


## ▸ TESTE 1: Carregar Dados Pós-Correção

<div style="background-color: #2d2416; border-left: 4px solid #f59e0b; padding: 15px; border-radius: 4px;">

**ESCOPO**

Carregar datasets regenerados após correções de feature engineering para validação de integridade.

</div>

In [2]:
# Carregar dados recém gerados (sem leakage)
X_train_new = pd.read_csv(DATA_DIR / 'X_train_engineered.csv')
X_test_new = pd.read_csv(DATA_DIR / 'X_test_engineered.csv')
y_train_new = pd.read_csv(DATA_DIR / 'y_train_engineered.csv').squeeze()
y_test_new = pd.read_csv(DATA_DIR / 'y_test_engineered.csv').squeeze()

print(f"Train shape: {X_train_new.shape}")
print(f"Test shape: {X_test_new.shape}")
print(f"Train fraud rate: {y_train_new.mean():.4f}")
print(f"Test fraud rate: {y_test_new.mean():.4f}")
print(f"\nFeatures: {list(X_train_new.columns)}")

Train shape: (2400, 12)
Test shape: (1460, 12)
Train fraud rate: 0.2500
Test fraud rate: 0.0096

Features: ['Dest Account', 'Payment Format', 'From Bank', 'Account', 'Day', 'To Bank', 'Amount Received', 'Amount Paid', 'From Bank_freq', 'To Bank_freq', 'Account_freq', 'Dest Account_freq']


## ▸ TESTE 2: Adversarial Validation

<div style="background-color: #2d2416; border-left: 4px solid #f59e0b; padding: 15px; border-radius: 4px;">

**ESCOPO**

Verificar se modelo consegue distinguir entre treino e teste. AUC < 0.55 indica datasets indistinguíveis (sem leakage).

</div>

In [3]:
# Preparar dados para adversarial validation
# Pegar apenas features numéricas comuns
numeric_features = X_train_new.select_dtypes(include=[np.number]).columns.tolist()

X_train_adv = X_train_new[numeric_features].copy()
X_test_adv = X_test_new[numeric_features].copy()

# Criar labels: 0 = train, 1 = test
X_combined = pd.concat([
    X_train_adv.assign(source=0),
    X_test_adv.assign(source=1)
], ignore_index=True)

X_adv = X_combined.drop('source', axis=1)
y_adv = X_combined['source']

print(f"Adversarial dataset: {X_adv.shape}")
print(f"Numeric features: {len(numeric_features)}")

Adversarial dataset: (3860, 12)
Numeric features: 12


In [4]:
# Adversarial Validation com Random Forest
print("[SEARCH] Running Adversarial Validation (5-Fold CV)...\n")

rf_adv = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42,
    n_jobs=-1
)

cv_scores = cross_val_score(
    rf_adv, X_adv, y_adv,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

mean_auc = cv_scores.mean()
std_auc = cv_scores.std()

print(f" Adversarial AUC: {mean_auc:.4f} ± {std_auc:.4f}")
print(f"   Fold scores: {[f'{s:.4f}' for s in cv_scores]}")
print()

# Interpretação
if mean_auc < 0.55:
    print("[OK] EXCELENTE: AUC < 0.55 → Sem leakage detectável!")
    print("   Distribuições train/test são indistinguíveis.")
elif mean_auc < 0.60:
    print("[OK] BOM: 0.55 < AUC < 0.60 → Leakage mínimo/aceitável")
    print("   Pequenas diferenças naturais entre splits.")
elif mean_auc < 0.65:
    print("[WARNING]  MODERADO: 0.60 < AUC < 0.65 → Leakage moderado")
    print("   Ainda há alguma diferença detectável.")
else:
    print("[FAIL] SEVERO: AUC > 0.65 → Leakage ainda presente!")
    print("   Necessário investigar mais.")

[SEARCH] Running Adversarial Validation (5-Fold CV)...

 Adversarial AUC: 1.0000 ± 0.0000
   Fold scores: ['1.0000', '1.0000', '1.0000', '1.0000', '1.0000']

[FAIL] SEVERO: AUC > 0.65 → Leakage ainda presente!
   Necessário investigar mais.


## ▸ TESTE 3: Análise de Correlation Gaps

<div style="background-color: #2d2416; border-left: 4px solid #f59e0b; padding: 15px; border-radius: 4px;">

**ESCOPO**

Comparar correlações feature-target entre treino e teste. Gaps altos indicam features com leakage.

</div>

In [5]:
# Calcular correlation gaps para features frequentes
print(" Correlation Gap Analysis\n")
print("Feature" + " " * 15 + "Train Corr  Test Corr  Gap      Ratio")
print("=" * 70)

freq_features = [col for col in X_train_new.columns if '_freq' in col]

for feature in freq_features:
    if feature in X_train_new.columns and feature in X_test_new.columns:
        # Correlation com target
        train_corr = X_train_new[feature].corr(y_train_new)
        test_corr = X_test_new[feature].corr(y_test_new)
        
        gap = abs(train_corr - test_corr)
        ratio = abs(train_corr / (test_corr + 1e-10))
        
        status = "[OK]" if gap < 0.10 and ratio < 5 else ("[WARNING]" if gap < 0.20 else "[FAIL]")
        
        print(f"{status} {feature:20s} {train_corr:7.4f}   {test_corr:7.4f}   {gap:6.4f}   {ratio:5.1f}x")

print("\n· Interpretação:")
print("   [OK] Gap < 0.10 e Ratio < 5x: Sem leakage")
print("   [WARNING]  Gap < 0.20 e Ratio < 10x: Leakage mínimo")
print("   [FAIL] Gap > 0.20 ou Ratio > 10x: Leakage presente")

 Correlation Gap Analysis

Feature               Train Corr  Test Corr  Gap      Ratio
[FAIL] From Bank_freq        0.2566    0.0214   0.2352    12.0x
[OK] To Bank_freq          0.1479    0.0631   0.0848     2.3x
[WARNING] Account_freq          0.1674    0.0308   0.1366     5.4x
[OK] Dest Account_freq    -0.1248   -0.0294   0.0954     4.2x

· Interpretação:
   [OK] Gap < 0.10 e Ratio < 5x: Sem leakage
   [WARNING]  Gap < 0.20 e Ratio < 10x: Leakage mínimo
   [FAIL] Gap > 0.20 ou Ratio > 10x: Leakage presente


## ▸ TESTE 4: Performance de Generalização

<div style="background-color: #2d2416; border-left: 4px solid #f59e0b; padding: 15px; border-radius: 4px;">

**ESCOPO**

Treinar modelo rápido e verificar se há generalização adequada (test PR-AUC > 0.20, gap < 0.40).

</div>

In [6]:
# Treinar modelo rápido para verificar generalização
from sklearn.metrics import average_precision_score, roc_auc_score
from lightgbm import LGBMClassifier

print("· Treinando LightGBM rápido para validar generalização...\n")

model = LGBMClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    verbose=-1
)

model.fit(X_train_new, y_train_new)

# Predições
y_train_proba = model.predict_proba(X_train_new)[:, 1]
y_test_proba = model.predict_proba(X_test_new)[:, 1]

# Métricas
train_pr_auc = average_precision_score(y_train_new, y_train_proba)
test_pr_auc = average_precision_score(y_test_new, y_test_proba)
train_roc_auc = roc_auc_score(y_train_new, y_train_proba)
test_roc_auc = roc_auc_score(y_test_new, y_test_proba)

pr_gap = train_pr_auc - test_pr_auc
roc_gap = train_roc_auc - test_roc_auc

print(f" Métricas:")
print(f"   Train PR-AUC:  {train_pr_auc:.4f}")
print(f"   Test PR-AUC:   {test_pr_auc:.4f}")
print(f"   Gap:           {pr_gap:.4f} ({pr_gap/train_pr_auc*100:.1f}%)")
print()
print(f"   Train ROC-AUC: {train_roc_auc:.4f}")
print(f"   Test ROC-AUC:  {test_roc_auc:.4f}")
print(f"   Gap:           {roc_gap:.4f} ({roc_gap/train_roc_auc*100:.1f}%)")
print()

# Avaliação
if test_pr_auc > 0.30 and pr_gap < 0.30:
    print("[OK] SUCESSO: Test PR-AUC > 0.30 e gap < 0.30")
    print("   Modelo generalizou bem! Leakage corrigido.")
elif test_pr_auc > 0.20 and pr_gap < 0.40:
    print("[OK] BOM: Test PR-AUC > 0.20 e gap < 0.40")
    print("   Generalização aceitável. Ainda há margem de melhoria.")
elif test_pr_auc > 0.05:
    print("[WARNING]  MODERADO: Test PR-AUC > 0.05")
    print("   Modelo aprende algo, mas precisa tuning.")
else:
    print("[FAIL] PROBLEMA: Test PR-AUC < 0.05")
    print("   Modelo não está generalizando. Investigar.")

· Treinando LightGBM rápido para validar generalização...

 Métricas:
   Train PR-AUC:  0.9996
   Test PR-AUC:   0.1726
   Gap:           0.8270 (82.7%)

   Train ROC-AUC: 0.9999
   Test ROC-AUC:  0.9421
   Gap:           0.0578 (5.8%)

[WARNING]  MODERADO: Test PR-AUC > 0.05
   Modelo aprende algo, mas precisa tuning.


---

## CONCLUSÃO E VEREDICTO FINAL

### RESUMO DOS TESTES EXECUTADOS

In [7]:
print("="*70)
print(" RESUMO DA VALIDAÇÃO DE CORREÇÃO DE LEAKAGE")
print("="*70)
print()
print(f"1. Adversarial AUC:  {mean_auc:.4f} {'[OK]' if mean_auc < 0.60 else '[FAIL]'}")
print(f"2. Test PR-AUC:      {test_pr_auc:.4f} {'[OK]' if test_pr_auc > 0.20 else '[FAIL]'}")
print(f"3. Train-Test Gap:   {pr_gap:.4f} ({pr_gap/train_pr_auc*100:.1f}%) {'[OK]' if pr_gap < 0.40 else '[FAIL]'}")
print()
print("Conclusão:")
if mean_auc < 0.60 and test_pr_auc > 0.20 and pr_gap < 0.40:
    print("[OK] LEAKAGE CORRIGIDO COM SUCESSO!")
    print("   - Adversarial validation passou")
    print("   - Modelo generaliza adequadamente")
    print("   - Gap de performance aceitável")
    print()
    print("· Próximo passo: Re-executar notebook 04 (model_development)")
elif mean_auc < 0.65 and test_pr_auc > 0.10:
    print("[WARNING]  MELHORIA PARCIAL")
    print("   - Leakage reduzido mas não eliminado")
    print("   - Verificar features suspeitas")
else:
    print("[FAIL] LEAKAGE AINDA PRESENTE")
    print("   - Necessário revisar encoding novamente")
print("="*70)

 RESUMO DA VALIDAÇÃO DE CORREÇÃO DE LEAKAGE

1. Adversarial AUC:  1.0000 [FAIL]
2. Test PR-AUC:      0.1726 [FAIL]
3. Train-Test Gap:   0.8270 (82.7%) [FAIL]

Conclusão:
[FAIL] LEAKAGE AINDA PRESENTE
   - Necessário revisar encoding novamente


<table>
<tr><th>Teste</th><th>Métrica</th><th>Threshold</th><th>Resultado</th><th>Status</th></tr>
<tr><td>1</td><td>Adversarial AUC</td><td>< 0.60</td><td>Ver output acima</td><td>-</td></tr>
<tr><td>2</td><td>Test PR-AUC</td><td>> 0.20</td><td>Ver output acima</td><td>-</td></tr>
<tr><td>3</td><td>Train-Test Gap</td><td>< 0.40</td><td>Ver output acima</td><td>-</td></tr>
<tr><td>4</td><td>Correlation Gaps</td><td>< 5x</td><td>Ver Teste 3</td><td>-</td></tr>
</table>

### INTERPRETAÇÃO DOS RESULTADOS

<div style="background-color: #16291c; border-left: 4px solid #10b981; padding: 15px; border-radius: 4px;">

**[OK] LEAKAGE CORRIGIDO (SE TODOS OS TESTES PASSARAM):**

- Train e test são indistinguíveis (Adversarial AUC baixo)
- Modelo generaliza para dados novos (Test PR-AUC razoável)
- Overfitting controlado (Gap < 40%)
- Features consistentes entre splits

**PRÓXIMAS AÇÕES:**
1. Prosseguir para notebook 06 (feature engineering avançado)
2. Re-executar notebook 07 (model development) com dados limpos
3. Documentar correções aplicadas

</div>

<div style="background-color: #2d2416; border-left: 4px solid #f59e0b; padding: 15px; border-radius: 4px; margin-top: 15px;">

**[WARNING] MELHORIA PARCIAL (SE ALGUNS TESTES FALHARAM):**

- Leakage reduzido mas não eliminado completamente
- Modelo está melhor mas ainda há espaço para melhoria
- Algumas features podem ter leakage residual

**AÇÕES CORRETIVAS:**
1. Revisar features com correlation gap > 5x
2. Verificar se encoding foi aplicado corretamente
3. Considerar aumentar temporal gap entre train/test
4. Re-auditar features agregadas

</div>

<div style="background-color: #2d1a1a; border-left: 4px solid #ef4444; padding: 15px; border-radius: 4px; margin-top: 15px;">

**[FAIL] LEAKAGE AINDA PRESENTE (SE TESTES CRÍTICOS FALHARAM):**

- Adversarial AUC alto (> 0.65) indica forte leakage
- Test PR-AUC muito baixo indica modelo não generaliza
- Gap alto indica overfitting severo

**AÇÕES URGENTES:**
1. Retornar ao notebook 03 (feature audit)
2. Revisar completamente pipeline de encoding
3. Validar split temporal (notebook 01)
4. Considerar remover features problemáticas
5. Re-executar validação após correções

</div>

---

<div align="center">

**EXECUTION TIME:** ~5 min | **VERSION:** 1.0 | **LAST UPDATED:** Oct 2025

![Validation](https://img.shields.io/badge/Validation-Complete-success)
![Quality](https://img.shields.io/badge/Quality-Assured-brightgreen)

</div>